# IRIDE

$\theta$ = 90°

nota, i valori sono approssimativi $\pm$ 20kHz 

$\nu$ = frequenza fotoni rilevati

In [7]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path)
print(dir_path)
from labbiofisica import Interpolazione, final_val

c:\Users\jocar\Desktop\LaboratorioBiofisica\SCATTERING


In [23]:
# Constants
n = 1.33 # index of refraction
η = 10**-9  # g/(nm*s) viscosity of water
λ = 633  # nm wavelength of the laser
T = 293  # K temperature
KB = 1.3806e-2  # g (nm)^2 / s^2 K boltzmann constant

sigmaT = 1 
sigmaη = 0.01*10**-9
sigman = 0.01

In [18]:
# Carico i dati {f: {t: [], g: [], sigmag: []}}
θ = 90
data = {}

IRIDE = [25, 50, 200, 300]   # f = frequenze delle varie aperture
GAMMA = []
sigmaGAMMA = []

# t convertito in secondi, si considerano dati solo tra 0.1 e 10 ms a causa del rumore
for 𝜈 in IRIDE: 
    filename = f'./data2/g_iride_{𝜈}_2.asc'
    
    read = pd.read_csv(filename, skiprows=30, nrows=245 - 30, sep='\t', encoding='ISO-8859-1')

    # print(read.columns[2:12])

    read = read[(read['Lag [ms]'] >= 0.1) & (read['Lag [ms]'] <= 10)]
    read['std'] = read.loc[:, read.columns[2:12]].std(axis=1) # converto in secondi
    
    t = read['Lag [ms]'].to_numpy()/1000
    g = read['Average'].to_numpy()
    σg = read['std'].to_numpy()/np.sqrt(9)
    
    data[𝜈] = {'t':t,'g':g,'sigmag': σg} # converto in secondi

IRIDE = np.array(IRIDE)
#print(data)

In [19]:
# fit data
CHI_r = []
DOF = []
PVALUE = []

f = lambda x, A, B, γ: A*np.exp(-γ*x) + B

for key in data.keys():
    x = data[key]['t']
    y = data[key]['g']
    sigmay = data[key]['sigmag']
    m = Interpolazione(x, y, sigmay, f, [0.05, 0.001, 2.7],['A','B','γ'])
    data[key]['fit'] = m
    # print(m.values['γ'])
    data[key]['gamma'] = m.values['γ']
    GAMMA.append(m.values['γ'])
    sigmaGAMMA.append(m.errors['γ'])
    CHI_r.append(m.rchi2)
    DOF = m.dof
    PVALUE.append(m.pvalue)

GAMMA = np.array(GAMMA) 
sigmaGAMMA = np.array(sigmaGAMMA)

In [20]:
# print FIT RESULT:
GAMMAFVAL = [final_val(g,sg,2,3,'Hz') for g,sg in zip(GAMMA,sigmaGAMMA)]
gammateta = pd.DataFrame({'𝜈':IRIDE,'γ':GAMMA,'σγ':sigmaGAMMA,'γ_out':GAMMAFVAL,'pvalue':PVALUE,'chir':CHI_r,'dof':DOF})

print('---------------------------FIT RESULT----------------------------')
display(gammateta)

---------------------------FIT RESULT----------------------------


,𝜈,γ,σγ,γ_out,pvalue,chir,dof
0,25,2521.138258,16.636726,(2.52 ± 0.02)e3 Hz,0.31,1.09,50
1,50,2417.044958,12.056272,(2.42 ± 0.01)e3 Hz,0.00,2.69,50
2,200,2392.782710,10.522507,(2.39 ± 0.01)e3 Hz,0.01,1.56,50
3,300,2388.238952,9.749063,(2.39 ± 0.01)e3 Hz,0.00,2.68,50


In [37]:
colors = px.colors.sequential.Turbo

fig = go.Figure()
for i,𝜈 in enumerate(data.keys()):
    x = data[𝜈]['t']
    y = data[𝜈]['g']
    γ = data[𝜈]['gamma']
    sigmay = data[𝜈]['sigmag']
    X,Y = data[𝜈]['fit'].draw()

    # plt.plot(X,Y)
    # plt.xlim(0,1)
    # plt.show()

    fig.add_trace(go.Scatter(x=X, y=Y, mode='lines',line_color=colors[2 * i], showlegend=False,hoverinfo='skip'))

    fig.add_trace(go.Scatter(x=x,y=y, mode='markers', name=f'𝜈 = {𝜈} kHz',
        marker=dict(color = colors[2 * i]),
        error_y=dict( 
			type='data', 
			array = sigmay, 
			color = colors[2 * i],
			thickness = 1.5, 
			width = 3, 
		)))

fig.update_layout(
    xaxis_type = "log",
    yaxis_title = "G<sup>2</sup>(τ)",
    xaxis_title = "t (s)",
    xaxis = dict(range = [np.log10(0.0001), np.log10(0.010)]),
    width = 800,
    height = 600,
    plot_bgcolor = 'white',
    font = dict(
        size = 18,
        color = "Black"
    ),
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        traceorder='normal',
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='black',
        borderwidth=1
    )
    )

fig.update_xaxes(
    mirror = True,
    ticks = 'outside',
    showline = True,
    linecolor = 'black',
    gridcolor = 'lightgrey'
)
fig.update_yaxes(
    mirror = True,
    ticks = 'outside',
    showline = True,
    linecolor = 'black',
    gridcolor = 'lightgrey'
)

fig.show()

In [28]:
def calculate_R(gamma):
    R = (16 * np.pi * n**2 * T * KB * np.sin(np.radians(θ) / 2)**2) / (3 * η * λ**2 * gamma)
    return R

def calculate_sigma_R(gamma, sigma_gamma,sigmaT,sigmaη,sigman, theta = 90):
    theta_rad = np.radians(theta)
    term1 = (16 * np.pi * n**2 * T * KB * np.sin(theta_rad / 2)**2) / (3 * η * λ**2 * gamma**2) * sigma_gamma
    term2 = (16 * np.pi * n**2 * KB * np.sin(theta_rad / 2)**2) / (3 * η * λ**2 * gamma) * sigmaT
    term3 = (16 * np.pi * n**2 * T * KB * np.sin(theta_rad / 2)**2) / (3 * η**2 * λ**2 * gamma) * sigmaη
    term4 = (32 * np.pi * n * T * KB * np.sin(theta_rad / 2)**2) / (3 * η * λ**2 * gamma) * sigman
    sigma_R = np.sqrt(term1**2 + term2**2 + term3**2 + term4**2)
    return sigma_R

R_values = calculate_R(GAMMA)
sigma_R = calculate_sigma_R(GAMMA, sigmaGAMMA,sigmaT,sigmaη,sigman)

R = pd.DataFrame({'𝜈':IRIDE,'R':R_values,'σR':sigma_R,'fval':np.array([final_val(R,sigmaR,1,0,'nm') for R,sigmaR in zip(R_values,sigma_R)])})
display(R)

,𝜈,R,σR,fval
0,25,59.340786,1.158777,59.3 ± 1.2 nm
1,50,61.896377,1.178729,61.9 ± 1.2 nm
2,200,62.523991,1.181549,62.5 ± 1.2 nm
3,300,62.642947,1.179354,62.6 ± 1.2 nm


In [35]:
mean_R = np.average(R_values,weights=1/sigma_R**2)
sigma_mean_R = np.sqrt(1/np.sum(1/sigma_R**2))
print(f'R = {final_val(mean_R,sigma_mean_R,1,0,"nm")}')

# COMPATIBILITA'

from scipy.stats import norm

ts = []
perc = []
for R,σR in zip(R_values,sigma_R):
    σ = np.sqrt(σR**2 + sigma_mean_R**2)
    t = np.round(np.abs(R-mean_R)/σ,3)
    ts.append(t)
    perc.append(f"{np.round(norm.cdf(t)*100)}%")

compatibilità = pd.DataFrame({'𝜈':IRIDE,'t':ts,'perc':perc})
display(compatibilità)

R = 61.6 ± 0.6 nm


,𝜈,t,perc
0,25,1.724,96.0%
1,50,0.240,59.0%
2,200,0.715,76.0%
3,300,0.807,79.0%
